<a href="https://colab.research.google.com/github/minhaz1172/Deep-Learning-with-PyTorch/blob/main/CNN_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# upload kaggle.json()

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"c20c43ca9e6e5be6fa69c912fab503c3"}'}

# check GPU Availability

In [7]:

import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Move and Set Permission

In [8]:
import os
import zipfile

# Make directory and move kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set proper permissions
!chmod 600 ~/.kaggle/kaggle.json

# install kaggle package

In [9]:
!pip install -q kaggle

# find and download dataset

In [10]:
!kaggle datasets download -d zalando-research/fashionmnist

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
 99% 68.0M/68.8M [00:00<00:00, 710MB/s]
100% 68.8M/68.8M [00:00<00:00, 709MB/s]


# unzip the dataset

In [11]:
!unzip /content/fashionmnist.zip

Archive:  /content/fashionmnist.zip
  inflating: fashion-mnist_test.csv  
  inflating: fashion-mnist_train.csv  
  inflating: t10k-images-idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte  
  inflating: train-images-idx3-ubyte  
  inflating: train-labels-idx1-ubyte  


# install optuna


In [12]:
!pip install optuna
import optuna

# import necessRY LIABRARIES

In [13]:
import numpy as np
import pandas as pd
import torch


Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms

In [14]:
# SET random seeds for reproducibility
torch.manual_seed(42)

# Load csv fi;e

In [15]:
df=pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# sepate feature and values
# iloc[:, 1:] = select: ,All rows (:),From column 1 to the last column (1:), skipping column 0,These columns represent the 784 pixel values of each image (flattened),
#.values:
#Converts the selected DataFrame part to a NumPy array,So X will have shape: (60000, 784) → 60,000 images, 784 features each

In [16]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

# train test split

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Scaling Features
Each pixel value ranges from 0 to 255 (grayscale)

But neural networks work better when inputs are scaled between 0 and 1

So dividing by 255 does:

0 → 0.0

255 → 1.0

128 → 0.50196 ... and so on.

This converts all pixel values into the [0, 1] range.

# Now why reshape in CNN:
CNNs expect 4D input tensors:

CNNs expect 4D input tensors:
(N, C, H, W) = (batch_size, channels, height, width)
So:

For grayscale images (like MNIST): C = 1

For RGB images: C = 3

In [18]:
X_train = X_train / 255.0          # Normalization
X_test = X_test / 255.0            # Normalization

X_train = X_train.reshape(-1, 1, 28, 28)  # Reshape for CNN input
X_test = X_test.reshape(-1, 1, 28, 28)


# Data Augmentation

In [19]:
# train Data Augmentation for the traing and test dataset

from torchvision import transforms


train_trainsform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0,translate=(0.1,0.1)),
    transforms.ToTensor()

      ])

test_transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

# Dataset Class

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels,transform=None):
        self.features = torch.tensor(features, dtype=torch.float32)  # normalize already done
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.transform=transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        feature,labels= self.features[index], self.labels[index]
        if self.transform:
          feature=self.transform(feature.squeeze().numpy())
        return feature,labels

create a train dataset object

In [21]:
train_dataset=CustomDataset(X_train,y_train,transform=train_trainsform)

# Len of train dataset

In [22]:
len(train_dataset)

48000

# create test dataset

In [23]:
test_dataset=CustomDataset(X_test,y_test,transform=test_transform)

# create train and test loader

In [24]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [25]:
X_train.shape[1]

1

# Define CNN

 # nn.BatchNorm2d(32)
  #32 = number of channels (feature maps), not batch size!

  # It normalizes each of the 32 channels independently over the batch.

In [35]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, num_conv_layers, num_filters, kernel_size, num_fc_layers, fc_layer_size, dropout_rate,
                 weight_decay, learning_rate, optimizer, batch_size, num_epochs):
        super().__init__()

        ### 1. Convolutional layers
        layers = []
        in_channels = 1  # Grayscale images have 1 channel

        for _ in range(num_conv_layers):
            layers.append(nn.Conv2d(in_channels, num_filters, kernel_size=kernel_size, padding=1))
            layers.append(nn.BatchNorm2d(num_filters))  # corrected: BatchNorm2d, not Batchnorm2d
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = num_filters  # update for next conv layer

        self.features = nn.Sequential(*layers)

        ### 2. Fully connected layers
        fc_layers = [nn.Flatten()]

        # Calculate flattened size: assume input image is 28x28
        conv_output_size = 28 // (2 ** num_conv_layers)
        input_size = num_filters * conv_output_size * conv_output_size

        for _ in range(num_fc_layers):
            fc_layers.append(nn.Linear(input_size, fc_layer_size))  # corrected: nn.Linar → nn.Linear
            fc_layers.append(nn.BatchNorm1d(fc_layer_size))
            fc_layers.append(nn.ReLU())
            fc_layers.append(nn.Dropout(dropout_rate))
            input_size = fc_layer_size  # update for next FC layer

        # Final output layer for 10 classes
        fc_layers.append(nn.Linear(input_size, 10))

        self.classifier = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Define the objective function for optuna

In [36]:
def objective(trial):

  # hyperparameter tunning

  num_conv_layers=trial.suggest_int('num_conv_layers',1,3)
  num_filters=trial.suggest_categorical('num_filters',[16,32,64,128])
  kernel_size=trial.suggest_categorical('kernel_size',[3,5])
  num_fc_layers=trial.suggest_int('num_fc_layers',1,3)
  fc_layers_size=trial.suggest_categorical('fc_layers_size',[64,128,256])
  dropout_rate=trial.suggest_float("dropout_rate",0.2,0.5)
  weight_decay=trial.suggest_float('weight_decay',1e-4,1e-2, log=True)
  learning_rate=trial.suggest_float("learning_rate",1e-4,1e-2, log=True)
  optimizer_name=trial.suggest_categorical('optimzer',['SGD','Adam','RMSprop'])
  batch_size=trial.suggest_categorical('batch_size',[32,64,128])
  num_epochs=trial.suggest_int('epochs',10,30)


# Model
  model=CNN(num_conv_layers,num_filters,kernel_size,num_fc_layers,fc_layers_size,dropout_rate,weight_decay,learning_rate,optimizer_name,batch_size,num_epochs)
  model.to(device)

  # data
  train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,pin_memory=True)
  test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)


  #optimizer

  if optimizer_name=='SGD':
    optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

  elif optimizer_name=='Adam':
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

  else:
    optimizer=torch.optim.RMSprop(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

    # loss function

    criterion=nn.CrossEntropyLoss()




    # trainig loop

    loss_history=[]

    for epoch in range(num_epochs):
      model.train()
      total_epoch_loss=0

      for features,labels in train_loader:
        features,labels=features.to(device),labels.to(device)


        optimizer.zero_grad()
        output=model(features)
        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()

        total_epoch_loss=total_epoch_loss+loss.item()
        avg_loss=total_epoch_loss/len(train_loader)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch:{epoch+1},Loss:{avg_loss:.4f}")

            loss_history.append(loss.item())



# validation loop

      model.eval()

      correct=0
      total=0

      with torch.no_grad():

        for features,labels in test_loader:
          features,labels=features.to(device),labels.to(device)
          output=model(features)
          _,predicted=torch.max(output.data,1)
          total=total+labels.shape[0]
          correct=correct+(predicted==labels).sum().item()
          accuracy=100*correct/total
          print(f'Test Accuracy: {accuracy:.2f}%')

    return accuracy

# run the optuna study


In [37]:
study=optuna.create_study(direction='maximize',)
study.optimize(objective,n_trials=10)

[I 2025-08-06 18:55:21,388] A new study created in memory with name: no-name-5f5d2e93-21c8-468d-a335-98608378be35
[W 2025-08-06 18:55:21,406] Trial 0 failed with parameters: {'num_conv_layers': 2, 'num_filters': 64, 'kernel_size': 3, 'num_fc_layers': 1, 'fc_layers_size': 256, 'dropout_rate': 0.38365621161895547, 'weight_decay': 0.004216601870441002, 'learning_rate': 0.00020038587990088352, 'optimzer': 'Adam', 'batch_size': 128, 'epochs': 10} because of the following error: The value None could not be cast to float..
[W 2025-08-06 18:55:21,406] Trial 0 failed with value None.
[W 2025-08-06 18:55:21,419] Trial 1 failed with parameters: {'num_conv_layers': 2, 'num_filters': 64, 'kernel_size': 5, 'num_fc_layers': 3, 'fc_layers_size': 128, 'dropout_rate': 0.26466701211848265, 'weight_decay': 0.0001799862095184211, 'learning_rate': 0.00012734555499526842, 'optimzer': 'Adam', 'batch_size': 128, 'epochs': 29} because of the following error: The value None could not be cast to float..
[W 2025-0

Streaming output truncated to the last 5000 lines.
Test Accuracy: 88.53%
Test Accuracy: 88.54%
Test Accuracy: 88.52%
Test Accuracy: 88.53%
Test Accuracy: 88.53%
Test Accuracy: 88.52%
Test Accuracy: 88.52%
Test Accuracy: 88.53%
Test Accuracy: 88.54%
Test Accuracy: 88.51%
Test Accuracy: 88.51%
Test Accuracy: 88.51%
Test Accuracy: 88.52%
Test Accuracy: 88.51%
Test Accuracy: 88.52%
Test Accuracy: 88.53%
Test Accuracy: 88.55%
Test Accuracy: 88.58%
Test Accuracy: 88.55%
Test Accuracy: 88.59%
Test Accuracy: 88.59%
Test Accuracy: 88.55%
Test Accuracy: 88.50%
Test Accuracy: 88.50%
Test Accuracy: 88.49%
Test Accuracy: 88.48%
Test Accuracy: 88.47%
Test Accuracy: 88.47%
Test Accuracy: 88.45%
Test Accuracy: 88.42%
Test Accuracy: 88.43%
Test Accuracy: 88.44%
Test Accuracy: 88.44%
Test Accuracy: 88.42%
Test Accuracy: 88.39%
Test Accuracy: 88.40%
Test Accuracy: 88.41%
Test Accuracy: 88.38%
Test Accuracy: 88.33%
Test Accuracy: 88.29%
Test Accuracy: 88.33%
Test Accuracy: 88.35%
Test Accuracy: 88.39%
Tes

[I 2025-08-06 19:01:07,381] Trial 3 finished with value: 90.16666666666667 and parameters: {'num_conv_layers': 3, 'num_filters': 32, 'kernel_size': 3, 'num_fc_layers': 1, 'fc_layers_size': 256, 'dropout_rate': 0.2960075541204782, 'weight_decay': 0.0008896537481797791, 'learning_rate': 0.000864168939748278, 'optimzer': 'RMSprop', 'batch_size': 32, 'epochs': 13}. Best is trial 3 with value: 90.16666666666667.
[W 2025-08-06 19:01:07,388] Trial 4 failed with parameters: {'num_conv_layers': 2, 'num_filters': 32, 'kernel_size': 3, 'num_fc_layers': 3, 'fc_layers_size': 64, 'dropout_rate': 0.4475732175771677, 'weight_decay': 0.0011576973856114495, 'learning_rate': 0.00019320361084341605, 'optimzer': 'SGD', 'batch_size': 128, 'epochs': 11} because of the following error: The value None could not be cast to float..
[W 2025-08-06 19:01:07,389] Trial 4 failed with value None.
[W 2025-08-06 19:01:07,396] Trial 5 failed with parameters: {'num_conv_layers': 2, 'num_filters': 16, 'kernel_size': 5, 'nu

Test Accuracy: 90.12%
Test Accuracy: 90.11%
Test Accuracy: 90.14%
Test Accuracy: 90.15%
Test Accuracy: 90.12%
Test Accuracy: 90.14%
Test Accuracy: 90.13%
Test Accuracy: 90.14%
Test Accuracy: 90.14%
Test Accuracy: 90.12%
Test Accuracy: 90.12%
Test Accuracy: 90.11%
Test Accuracy: 90.09%
Test Accuracy: 90.11%
Test Accuracy: 90.12%
Test Accuracy: 90.14%
Test Accuracy: 90.14%
Test Accuracy: 90.15%
Test Accuracy: 90.14%
Test Accuracy: 90.15%
Test Accuracy: 90.15%
Test Accuracy: 90.17%
Test Accuracy: 90.18%
Test Accuracy: 90.17%
Test Accuracy: 90.17%
Test Accuracy: 90.18%
Test Accuracy: 90.18%
Test Accuracy: 90.19%
Test Accuracy: 90.19%
Test Accuracy: 90.17%


[W 2025-08-06 19:01:07,412] Trial 6 failed with parameters: {'num_conv_layers': 1, 'num_filters': 64, 'kernel_size': 5, 'num_fc_layers': 1, 'fc_layers_size': 64, 'dropout_rate': 0.23820657743232282, 'weight_decay': 0.004188142164635526, 'learning_rate': 0.00029319734309671383, 'optimzer': 'Adam', 'batch_size': 32, 'epochs': 11} because of the following error: The value None could not be cast to float..
[W 2025-08-06 19:01:07,412] Trial 6 failed with value None.
[W 2025-08-06 19:01:07,423] Trial 7 failed with parameters: {'num_conv_layers': 3, 'num_filters': 128, 'kernel_size': 3, 'num_fc_layers': 3, 'fc_layers_size': 64, 'dropout_rate': 0.2235311283578639, 'weight_decay': 0.0022922348146242714, 'learning_rate': 0.0003496692576074113, 'optimzer': 'SGD', 'batch_size': 32, 'epochs': 28} because of the following error: The value None could not be cast to float..
[W 2025-08-06 19:01:07,424] Trial 7 failed with value None.
[W 2025-08-06 19:01:07,443] Trial 8 failed with parameters: {'num_con

# check best accuracy and best params

In [38]:
study.best_value

90.16666666666667

In [39]:
study.best_params

{'num_conv_layers': 3,
 'num_filters': 32,
 'kernel_size': 3,
 'num_fc_layers': 1,
 'fc_layers_size': 256,
 'dropout_rate': 0.2960075541204782,
 'weight_decay': 0.0008896537481797791,
 'learning_rate': 0.000864168939748278,
 'optimzer': 'RMSprop',
 'batch_size': 32,
 'epochs': 13}